# Neural Data Analysis: Spike Trains and Population Coding

## 🎯 Learning Objectives

By the end of this tutorial, you will be able to:

- Generate synthetic spike trains using Poisson processes.
- Visualize neural activity using raster plots.
- Calculate basic statistics like firing rates and Inter-Spike Intervals (ISIs).
- Compute variability measures such as the Coefficient of Variation (CV) and Fano Factor.
- Apply Information Theory concepts like Entropy and Mutual Information to neural data.
- Implement a population decoder to classify stimuli based on neural activity.
- Use standard Machine Learning libraries (`scikit-learn`) for neural decoding.
- Understand the difference between single-neuron and population coding.

## 📚 Prerequisites

- Basic Python programming (lists, loops, functions).
- Familiarity with `numpy` arrays.
- Basic understanding of probability (distributions, means).

## Introduction

Neurons communicate using discrete electrical pulses called **action potentials** or **spikes**. A sequence of spikes over time is called a **spike train**. Because the exact timing of spikes can be irregular, we often treat them as stochastic processes.

In this tutorial, we will explore how to analyze these spike trains. We will start by generating synthetic data, then move on to visualizing it, calculating statistics, and finally, using the activity of a population of neurons to "decode" what stimulus the brain is seeing.

## Setup and Imports


In [ ]:
import doctest

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA

from neuroai import plotting

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
def generate_spike_train(rate: float, duration: float) -> np.ndarray:
    """
    Generates spike times for a Poisson process.

    Args:
        rate (float): Firing rate in Hz.
        duration (float): Duration in seconds.

    Returns:
        np.array: Array of spike times.
    """
    if rate <= 0:
        return np.array([])

    # Generate ISIs from an exponential distribution
    # We generate a buffer of spikes to ensure we cover the full duration
    expected_spikes = int(rate * duration * 2.0) + 10
    isis = np.random.exponential(1.0 / rate, expected_spikes)
    spike_times = np.cumsum(isis)

    # Keep only spikes within the duration
    return spike_times[spike_times < duration]

In [ ]:
# Parameters
n_neurons = 20
n_trials = 50
duration = 1.0  # seconds

# Define firing rates for two stimuli
# We make the difference between stimuli subtle to highlight the benefit of population coding.
# Stimulus A: Neurons 0-9 have slightly higher rates
# Stimulus B: Neurons 10-19 have slightly higher rates
base_rate = 15.0  # Hz
delta_rate = 5.0  # Hz difference

rates_A = np.ones(n_neurons) * base_rate
rates_A[:10] += delta_rate  # Neurons 0-9 fire at 20Hz

rates_B = np.ones(n_neurons) * base_rate
rates_B[10:] += delta_rate  # Neurons 10-19 fire at 20Hz

# Generate data structure: data[stimulus][trial][neuron_idx] = spike_times_array
data = {"A": [], "B": []}

for trial in range(n_trials):
    # Stimulus A trials
    trial_spikes_A = []
    for n in range(n_neurons):
        trial_spikes_A.append(generate_spike_train(rates_A[n], duration))
    data["A"].append(trial_spikes_A)

    # Stimulus B trials
    trial_spikes_B = []
    for n in range(n_neurons):
        trial_spikes_B.append(generate_spike_train(rates_B[n], duration))
    data["B"].append(trial_spikes_B)

print(f"Generated data for {n_neurons} neurons, {n_trials} trials per stimulus.")

## 2. Visualizing Raster Plots

Before analysing, it's always good to look at the data. A raster plot shows spike times for each neuron in a single trial.


In [ ]:
# Plot one trial from Stimulus A
plotting.plot_raster_from_list(data["A"][0], "Raster Plot - Stimulus A (Trial 0)")

# Plot one trial from Stimulus B
plotting.plot_raster_from_list(data["B"][0], "Raster Plot - Stimulus B (Trial 0)")

### 🧠 Let's think about it!

Look at the raster plots above.

1. Can you see a difference between Stimulus A and Stimulus B just by eye?
2. Are there any neurons that seem to fire for both stimuli?
3. How variable is the firing? Do the spikes look perfectly regular (like a clock) or random?


## 3. Calculating Firing Rates

The firing rate is the number of spikes per unit time (usually spikes/second or Hz).

### 🎓 Exercise 1: Calculate Firing Rate

**Task:**
Write a function to calculate the average firing rate of a neuron across a trial.


In [ ]:
def calculate_firing_rate(spike_times: np.ndarray, duration: float) -> float:
    """
    Calculates firing rate in Hz.

    Args:
        spike_times (np.array): Array of spike times in seconds.
        duration (float): Duration of the recording in seconds.

    Returns:
        float: Firing rate in Hz.

    Examples:
        >>> spikes = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
        >>> calculate_firing_rate(spikes, 1.0)
        5.0
        >>> calculate_firing_rate(spikes, 0.5)
        10.0
        >>> calculate_firing_rate(np.array([]), 1.0)
        0.0
    """
    ################################
    # YOUR CODE HERE
    # Hint: Firing rate = number of spikes / duration
    ################################
    pass


# Test your implementation
doctest.run_docstring_examples(calculate_firing_rate, globals())

### ✅ Solution - Exercise 1

<details>
<summary>Click to reveal solution</summary>

```python
def calculate_firing_rate(spike_times: np.ndarray, duration: float) -> float:
    num_spikes = len(spike_times)
    return num_spikes / duration
```

</details>


## 4. Inter-Spike Intervals (ISIs)

The Inter-Spike Interval (ISI) is the time difference between consecutive spikes. The distribution of ISIs can tell us about the regularity of firing.

### 🎓 Exercise 2: Calculate ISIs

**Task:**
Calculate the ISIs for a specific neuron across all trials of Stimulus A and plot the histogram.


In [ ]:
def get_isis(spike_times: np.ndarray) -> np.ndarray:
    """
    Calculates inter-spike intervals (time between consecutive spikes).

    Args:
        spike_times (np.array): Sorted spike times.

    Returns:
        np.array: Array of ISIs.

    Examples:
        >>> spikes = np.array([0.1, 0.2, 0.35, 0.6])
        >>> get_isis(spikes)
        array([0.1 , 0.15, 0.25])
        >>> get_isis(np.array([0.5]))  # Single spike has no ISIs
        array([], dtype=float64)
        >>> get_isis(np.array([]))  # No spikes
        array([], dtype=float64)
    """
    ################################
    # YOUR CODE HERE
    # Hint: Use np.diff() to compute differences between consecutive elements
    ################################
    pass


# Test your implementation
doctest.run_docstring_examples(get_isis, globals())

In [ ]:
all_isis = []
neuron_idx = 0
for trial in data["A"]:
    spikes = trial[neuron_idx]
    isis = get_isis(spikes)
    all_isis.extend(isis)

fig = px.histogram(
    x=all_isis, nbins=50, labels={"x": "ISI (s)", "y": "Count"}, title="ISI Distribution"
)
fig.show()

### 🧠 Let's think about it!

1. What is the shape of the ISI distribution? Does it look like an exponential distribution?
2. What does an exponential ISI distribution imply about the underlying spike generation process? (Hint: Think about Poisson processes).
3. If the neuron was firing perfectly regularly (like a metronome), what would the ISI histogram look like?


## 5. Variability Statistics

Neurons in the brain are often noisy. Two common measures to quantify this variability are the **Coefficient of Variation (CV)** of the ISIs and the **Fano Factor** of the spike counts.

### Coefficient of Variation (CV)

The CV measures the irregularity of spike timing. It is defined as the ratio of the standard deviation of the ISIs to the mean ISI:

$$ CV = \frac{\sigma*{ISI}}{\mu*{ISI}} $$

- **CV ≈ 0**: Extremely regular firing (like a clock).
- **CV ≈ 1**: Poisson process (random spikes).
- **CV > 1**: Bursty firing.

**Read more:** [Coefficient of variation (Wikipedia)](https://en.wikipedia.org/wiki/Coefficient_of_variation)

### Fano Factor

The Fano Factor measures the variability of the spike count over repeated trials. It is defined as the variance of the spike count divided by the mean spike count:

$$ F = \frac{\sigma^2*{counts}}{\mu*{counts}} $$

- **F ≈ 1**: Poisson process.
- **F < 1**: Regular firing.
- **F > 1**: Highly variable (bursty or doubly stochastic).

**Read more:** [Fano factor (Wikipedia)](https://en.wikipedia.org/wiki/Fano_factor)

### 🎓 Exercise 3: Calculate CV and Fano Factor

**Task:**

1.  Write a function to calculate the CV of a spike train.
2.  Write a function to calculate the Fano Factor for a neuron across multiple trials.
3.  Calculate these values for our generated data.


In [ ]:
def calculate_cv(isis: np.ndarray) -> float:
    """
    Calculates the Coefficient of Variation (CV) of ISIs.

    CV = std(ISIs) / mean(ISIs)
    - CV ≈ 0: Regular firing
    - CV ≈ 1: Poisson process
    - CV > 1: Bursty firing

    Args:
        isis (np.array): Array of Inter-Spike Intervals.

    Returns:
        float: CV value.

    Examples:
        >>> isis = np.array([0.05, 0.05, 0.05, 0.05])  # Regular firing
        >>> calculate_cv(isis)
        np.float64(0.0)
        >>> isis = np.array([0.1, 0.2, 0.3, 0.4])  # Variable ISIs
        >>> round(calculate_cv(isis), 4)
        np.float64(0.4472)
        >>> calculate_cv(np.array([0.1]))  # Single ISI
        np.float64(0.0)
    """
    ################################
    # YOUR CODE HERE
    # Hint: CV = standard deviation / mean
    ################################
    pass


def calculate_fano_factor(spike_counts: np.ndarray) -> float:
    """
    Calculates the Fano Factor of spike counts across trials.

    Fano Factor = variance(counts) / mean(counts)
    - F ≈ 1: Poisson process
    - F < 1: Regular firing
    - F > 1: Bursty firing

    Args:
        spike_counts (np.array): Array of spike counts across trials.

    Returns:
        float: Fano Factor.

    Examples:
        >>> counts = np.array([10, 10, 10, 10])  # No variability
        >>> calculate_fano_factor(counts)
        np.float64(0.0)
        >>> counts = np.array([8, 10, 12, 10])  # Some variability
        >>> calculate_fano_factor(counts)
        np.float64(0.2)
        >>> counts = np.array([5, 15, 5, 15])  # High variability
        >>> calculate_fano_factor(counts)
        np.float64(2.5)
    """
    ################################
    # YOUR CODE HERE
    # Hint: Fano Factor = variance / mean
    ################################
    pass


# Test your implementations
doctest.run_docstring_examples(calculate_cv, globals())
doctest.run_docstring_examples(calculate_fano_factor, globals())

## 6. Information Theory

Information theory provides a powerful framework for quantifying how much information a neuron carries about a stimulus.

### Entropy

Entropy ($H$) measures the uncertainty or "surprise" of a random variable. For a discrete random variable $X$ with probability mass function $P(x)$:

<!-- prettier-ignore -->
$$ H(X) = - \sum_{x} P(x) \log_2 P(x) $$

### Mutual Information

Mutual Information ($MI$) measures how much knowing one variable reduces uncertainty about another. In our case, we want to know how much information the **spike count** ($R$) carries about the **stimulus** ($S$).

$$ MI(S; R) = H(R) - H(R|S) $$

Where:

- $H(R)$ is the total entropy of the response (spike counts).
- $H(R|S)$ is the conditional entropy of the response given the stimulus (noise entropy).

### 🎓 Exercise 4: Calculate Mutual Information

**Task:**

1.  Discretize the spike counts for a single neuron (e.g., Neuron 0).
2.  Calculate the probability distribution of spike counts $P(r)$.
3.  Calculate the conditional probability distributions $P(r|s)$ for each stimulus.
4.  Compute the Mutual Information.

_Hint: You can use `np.histogram` to estimate probabilities._


In [ ]:
def calculate_entropy(prob_dist: np.ndarray) -> float:
    """
    Calculates Shannon entropy in bits.

    Entropy measures the uncertainty or "surprise" of a probability distribution.
    H(X) = -sum(P(x) * log2(P(x)))

    Args:
        prob_dist (np.array): Probability distribution (must sum to 1).

    Returns:
        float: Entropy in bits.

    Examples:
        >>> p = np.array([0.5, 0.5])  # Maximum entropy for 2 outcomes
        >>> calculate_entropy(p)
        1.0
        >>> p = np.array([1.0, 0.0])  # No uncertainty (zero entropy)
        >>> calculate_entropy(p)
        0.0
        >>> p = np.array([0.25, 0.25, 0.25, 0.25])  # Uniform over 4
        >>> calculate_entropy(p)
        2.0
        >>> p = np.array([0.9, 0.1])  # Low entropy (high certainty)
        >>> round(calculate_entropy(p), 4)
        0.469
    """
    ################################
    # YOUR CODE HERE
    # Hint: Filter out zero probabilities to avoid log(0)
    # Use np.log2 for bits
    ################################
    pass


def calculate_mutual_information(counts_A: np.ndarray, counts_B: np.ndarray) -> float:
    """
    Calculates Mutual Information between Stimulus (A/B) and Response (spike counts).

    MI(S; R) = H(R) - H(R|S)

    Assumes stimuli are presented with equal probability P(A)=P(B)=0.5.

    Args:
        counts_A (np.array): Spike counts for stimulus A trials.
        counts_B (np.array): Spike counts for stimulus B trials.

    Returns:
        float: Mutual Information in bits.

    Examples:
        >>> # Non-overlapping responses = maximum information (1 bit)
        >>> counts_A = np.array([10, 10, 10, 10])
        >>> counts_B = np.array([20, 20, 20, 20])
        >>> calculate_mutual_information(counts_A, counts_B)
        1.0
        >>> # Identical responses = zero information
        >>> counts_A = np.array([15, 15, 15, 15])
        >>> counts_B = np.array([15, 15, 15, 15])
        >>> calculate_mutual_information(counts_A, counts_B)
        0.0
        >>> # Partial overlap = partial information (0.5 bits)
        >>> counts_A = np.array([10, 10, 11, 11])  # Half at 10, half at 11
        >>> counts_B = np.array([11, 11, 12, 12])  # Half at 11, half at 12
        >>> calculate_mutual_information(counts_A, counts_B)
        0.5
    """
    ################################
    # YOUR CODE HERE
    # Steps:
    # 1. Define bins for response histogram
    # 2. Calculate P(r|A) and P(r|B) using np.histogram with density=True
    # 3. Calculate P(r) = 0.5 * P(r|A) + 0.5 * P(r|B)
    # 4. Compute H(R), H(R|A), H(R|B)
    # 5. MI = H(R) - 0.5*H(R|A) - 0.5*H(R|B)
    ################################
    pass


# Test your implementations
doctest.run_docstring_examples(calculate_entropy, globals())
doctest.run_docstring_examples(calculate_mutual_information, globals())

## 7. Population Decoding

Can we determine which stimulus was presented just by looking at the population activity?

We will implement a simple **template matching** decoder.

1. Calculate the "template" population vector for each stimulus (average firing rates across training trials).
2. For a new "test" trial, calculate its population vector.
3. Compare the test vector to the templates (e.g., using Euclidean distance or correlation).
4. Assign the stimulus label of the closest template.

### 🎓 Exercise 5: Population Decoder

**Task:**
Split the data into training (first 40 trials) and testing (last 10 trials). Train the decoder and evaluate its accuracy.


In [ ]:
def get_population_vector(trial_data: list[np.ndarray], duration: float) -> np.ndarray:
    """
    Returns an array of firing rates for all neurons in a trial.

    This creates a "population vector" - a snapshot of activity across all neurons
    that can be used for decoding which stimulus was presented.

    Args:
        trial_data (list): List of spike time arrays for each neuron.
        duration (float): Duration of the trial in seconds.

    Returns:
        np.array: Array of firing rates (one per neuron).

    Examples:
        >>> trial = [np.array([0.1, 0.2]), np.array([0.5])]
        >>> get_population_vector(trial, 1.0)
        array([2., 1.])
        >>> trial = [np.array([0.1, 0.2, 0.3]), np.array([]), np.array([0.5, 0.6])]
        >>> get_population_vector(trial, 1.0)
        array([3., 0., 2.])
        >>> trial = [np.array([0.1, 0.2])]
        >>> get_population_vector(trial, 0.5)  # Shorter duration = higher rate
        array([4.])
    """
    ################################
    # YOUR CODE HERE
    # Hint: Loop through each neuron's spike times and calculate firing rate
    # Or use a list comprehension with len(spikes) / duration
    ################################
    pass


# Test your implementation
doctest.run_docstring_examples(get_population_vector, globals())

In [ ]:
# Split data
n_train = 40
train_A = data["A"][:n_train]
test_A = data["A"][n_train:]
train_B = data["B"][:n_train]
test_B = data["B"][n_train:]


# 1. Calculate Templates (Mean population vectors)
mean_vector_A = np.mean([get_population_vector(trial, duration) for trial in train_A], axis=0)
mean_vector_B = np.mean([get_population_vector(trial, duration) for trial in train_B], axis=0)


# 2. Decode Test Trials
correct = 0
total = 0

# Test with Stimulus A trials
for trial in test_A:
    vec = get_population_vector(trial, duration)
    dist_A = np.linalg.norm(vec - mean_vector_A)
    dist_B = np.linalg.norm(vec - mean_vector_B)
    if dist_A < dist_B:
        correct += 1
    total += 1

# Test with Stimulus B trials
for trial in test_B:
    vec = get_population_vector(trial, duration)
    dist_A = np.linalg.norm(vec - mean_vector_A)
    dist_B = np.linalg.norm(vec - mean_vector_B)
    if dist_B < dist_A:
        correct += 1
    total += 1

print(f"Decoding Accuracy: {correct / total * 100}%")

### Exploration

Now that you have a working decoder, try exploring the following:

1. **Vary the number of neurons**: How does decoding accuracy change if you only use 5 neurons? Or 1 neuron?
2. **Vary the noise**: In the data generation function, try making the firing rates of the two stimuli more similar (e.g., 20Hz vs 22Hz). How does this affect accuracy?
3. **Time window**: What if you only use the first 100ms of the trial? Does accuracy drop?
4. **Different Decoder**: Try using a correlation-based metric instead of Euclidean distance. Does it work better?


## 8. Decoding with Machine Learning

While our manual template matcher works, modern neuroscience relies heavily on standard Machine Learning libraries. Let's use `scikit-learn` to train a classifier.

We will use **Logistic Regression**, a linear classifier that learns a weight for each neuron.

### 🎓 Exercise 6: Scikit-Learn Decoder

**Task:**

1.  Prepare the data matrix `X` (n_samples, n_features) and labels `y` (n_samples).
    - `X`: Each row is a trial, each column is a neuron's firing rate.
    - `y`: 0 for Stimulus A, 1 for Stimulus B.
2.  Split into training and testing sets.
3.  Train a `LogisticRegression` model.
4.  Calculate accuracy.


In [ ]:
def prepare_data_for_sklearn(data: dict, duration: float) -> tuple[np.ndarray, np.ndarray]:
    """
    Prepares spike train data for scikit-learn classifiers.

    Converts spike train data into feature matrix X and label vector y.

    Args:
        data (dict): Dictionary with keys 'A' and 'B', each containing trial data.
        duration (float): Duration of each trial in seconds.

    Returns:
        tuple: (X, y) where X is (n_samples, n_features) and y is (n_samples,)
            - X: Each row is a trial's population vector (firing rates)
            - y: Labels (0 for stimulus A, 1 for stimulus B)

    Examples:
        >>> # Create minimal test data
        >>> test_data = {
        ...     'A': [[np.array([0.1, 0.2]), np.array([0.5])]],  # 1 trial, 2 neurons
        ...     'B': [[np.array([0.3]), np.array([0.4, 0.5, 0.6])]]  # 1 trial, 2 neurons
        ... }
        >>> X, y = prepare_data_for_sklearn(test_data, 1.0)
        >>> X.shape
        (2, 2)
        >>> list(y)
        [0, 1]
    """
    ################################
    # YOUR CODE HERE
    # 1. Create empty lists for X and y
    # 2. Loop through data["A"] trials, append population vectors to X, append 0 to y
    # 3. Loop through data["B"] trials, append population vectors to X, append 1 to y
    # 4. Convert to numpy arrays
    ################################
    pass


def train_and_evaluate_decoder(
    X: np.ndarray, y: np.ndarray, test_size: float = 0.2, random_state: int = 42
) -> tuple[float, np.ndarray]:
    """
    Trains a Logistic Regression decoder and returns accuracy and weights.

    Args:
        X (np.ndarray): Feature matrix (n_samples, n_features).
        y (np.ndarray): Labels (n_samples,).
        test_size (float): Fraction of data to use for testing.
        random_state (int): Random seed for reproducibility.

    Returns:
        tuple: (accuracy, weights) where accuracy is a float and weights is (n_features,)

    Examples:
        >>> np.random.seed(42)
        >>> # Create separable data
        >>> X = np.vstack([np.random.randn(10, 2) + [2, 0], np.random.randn(10, 2) + [-2, 0]])
        >>> y = np.array([0]*10 + [1]*10)
        >>> acc, weights = train_and_evaluate_decoder(X, y)
        >>> acc >= 0.5  # Should be better than chance
        True
        >>> len(weights)
        2
    """
    ################################
    # YOUR CODE HERE
    # 1. Split data using train_test_split
    # 2. Create and train LogisticRegression model
    # 3. Predict on test set and calculate accuracy
    # 4. Return accuracy and model weights (clf.coef_[0])
    ################################
    pass


# Test your implementations
doctest.run_docstring_examples(prepare_data_for_sklearn, globals())
doctest.run_docstring_examples(train_and_evaluate_decoder, globals())


# Use the functions
X, y = prepare_data_for_sklearn(data, duration)
accuracy, weights = train_and_evaluate_decoder(X, y)

print(f"\nLogistic Regression Accuracy: {accuracy * 100:.2f}%")

# Plot weights
fig = px.bar(
    x=range(n_neurons), y=weights, labels={"x": "Neuron", "y": "Weight"}, title="Decoder Weights"
)
fig.show()

## 9. Individual vs. Population Decoding

A key question in neuroscience is whether information is distributed across a population or encoded by specific single neurons. Let's compare the decoding performance of the entire population against the performance of individual neurons.

We will:

1. Iterate through each neuron.
2. Train a decoder using **only** that neuron's activity.
3. Evaluate its accuracy.
4. Compare the best single neuron to the whole population.


### 🧠 Let's think about it!

Look at the weights plot above.

1.  Which neurons have positive weights? Which have negative weights?
2.  Does this match how we generated the data? (Recall: Stimulus A had higher rates for neurons 0-9, Stimulus B for 10-19).
3.  Why might Logistic Regression be better (or worse) than our simple template matcher?


In [ ]:
individual_accuracies = []

for neuron_idx in range(n_neurons):
    # 1. Calculate Templates for this neuron
    # Extract firing rates for this neuron only
    rates_A_neuron = [get_population_vector(trial, duration)[neuron_idx] for trial in train_A]
    rates_B_neuron = [get_population_vector(trial, duration)[neuron_idx] for trial in train_B]

    mean_A = np.mean(rates_A_neuron)
    mean_B = np.mean(rates_B_neuron)

    # 2. Decode Test Trials
    correct_neuron = 0
    total_neuron = 0

    for trial in test_A:
        rate = get_population_vector(trial, duration)[neuron_idx]
        if abs(rate - mean_A) < abs(rate - mean_B):
            correct_neuron += 1
        total_neuron += 1

    for trial in test_B:
        rate = get_population_vector(trial, duration)[neuron_idx]
        if abs(rate - mean_B) < abs(rate - mean_A):
            correct_neuron += 1
        total_neuron += 1

    individual_accuracies.append(correct_neuron / total_neuron * 100)

# Plot results
fig = go.Figure()
fig.add_trace(go.Bar(x=list(range(n_neurons)), y=individual_accuracies, name="Individual Neurons"))

# Add population accuracy line
population_acc = correct / total * 100
fig.add_hline(
    y=population_acc, line_dash="dash", line_color="red", annotation_text="Population Accuracy"
)

fig.update_layout(
    title="Individual vs. Population Decoding Accuracy",
    xaxis_title="Neuron Index",
    yaxis_title="Accuracy (%)",
    yaxis_range=[0, 105],
)
fig.show()

## 10. Advanced: NeuroAI Representations

We have analyzed spikes in various ways. In modern NeuroAI, we often map these biological properties to concepts in Artificial Neural Networks (ANNs).

Let's explore these connections explicitly.

### 10.1 Rate Coding & ReLU

The **firing rate** ($r$) is the most common abstraction. In ANNs, the **ReLU** (Rectified Linear Unit) activation function $f(x) = \max(0, x)$ mimics the threshold-linear behavior of neurons: below a threshold, they are silent; above it, their rate increases linearly with input current.

Let's simulate a neuron's response to increasing input drive.


In [ ]:
inputs = np.linspace(-5, 10, 20)
rates = []
for inp in inputs:
    # Simulate a neuron with a threshold and gain
    # r = max(0, input)
    true_rate = max(0, inp * 5)  # Gain of 5
    # Generate spikes
    spikes = generate_spike_train(true_rate, duration=1.0)
    # Measure rate
    measured_rate = len(spikes) / 1.0
    rates.append(measured_rate)

fig = px.scatter(
    x=inputs,
    y=rates,
    labels={"x": "Input Drive", "y": "Measured Firing Rate (Hz)"},
    title="f-I Curve: The Biological ReLU",
)
fig.add_trace(
    go.Scatter(x=inputs, y=[max(0, x * 5) for x in inputs], mode="lines", name="Ideal ReLU")
)
fig.show()

### 10.2 Temporal Coding (Latency)

Spiking Neural Networks (SNNs) often use the **time of the first spike** to encode information. Stronger inputs cause a neuron to reach its threshold faster, resulting in a shorter latency. This allows the brain to make decisions very quickly, even before a full rate can be estimated.


In [ ]:
inputs = np.linspace(1, 20, 20)  # Positive inputs
latencies = []
for inp in inputs:
    rate = inp * 5
    spikes = generate_spike_train(rate, duration=1.0)
    if len(spikes) > 0:
        latencies.append(spikes[0])
    else:
        latencies.append(1.0)  # Max duration if no spike

fig = px.line(
    x=inputs,
    y=latencies,
    labels={"x": "Input Drive", "y": "Time to First Spike (s)"},
    title="Latency Coding: Stronger Input = Faster Response",
)
fig.show()

### 10.3 Neural Embeddings (PCA)

The **population vector** ($P$) can be thought of as an **embedding** in a high-dimensional space. Just like word embeddings in NLP, neural states live on a "manifold". We can use dimensionality reduction techniques like **PCA** to visualize this space.

Here, we project our 20-dimensional neural activity down to 2 dimensions. Notice how the two stimuli form distinct clusters.


In [ ]:
# We already have X (population vectors) and y (labels) from Section 8
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

fig = px.scatter(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    color=y.astype(str),
    labels={"x": "PC1", "y": "PC2", "color": "Stimulus"},
    title="Neural Manifold: PCA of Population Activity",
)
fig.show()

### 10.4 Sparsity

Brains are energy-constrained, so they use **sparse representations** ($a$)—only a few neurons are active at any time. This is similar to L1 regularization or sparse autoencoders in AI.

We can measure population sparsity using **Hoyer's measure**, which ranges from 0 (all neurons active equally) to 1 (only one neuron active).


In [ ]:
# Calculate population sparsity for each trial
# Hoyer's sparsity measure: (sqrt(n) - L1/L2) / (sqrt(n) - 1)
def hoyer_sparsity(x):
    n = len(x)
    if n == 0:
        return 0
    # Avoid division by zero if vector is all zeros
    if np.sum(np.abs(x)) == 0:
        return 1.0
    l1 = np.linalg.norm(x, 1)
    l2 = np.linalg.norm(x, 2)
    return (np.sqrt(n) - (l1 / l2)) / (np.sqrt(n) - 1)


sparsities = [hoyer_sparsity(trial_vec) for trial_vec in X]
mean_sparsity = np.mean(sparsities)

print(f"Mean Population Sparsity (Hoyer's): {mean_sparsity:.4f}")
fig = px.histogram(
    x=sparsities, nbins=20, title="Distribution of Population Sparsity", labels={"x": "Sparsity"}
)
fig.show()

### 10.5 Criticality & Information Maximization

A major hypothesis in NeuroAI is that the brain operates at a **critical point**—a phase transition between ordered and chaotic dynamics. Systems at criticality are thought to maximize information processing capabilities (dynamic range, memory, information transmission).

Let's test this hypothesis using a simple **Branching Process** model.

1.  Start with 1 active neuron.
2.  At each time step, an active neuron activates $k$ other neurons in the next step, where $k$ is drawn from a Poisson distribution with mean $\sigma$ (the **branching ratio**).
3.  The "avalanche" stops when no neurons are active.
4.  We measure the **avalanche size** (total number of spikes).

**Hypothesis:** The entropy of the avalanche size distribution (a proxy for information capacity) is maximized when $\sigma \approx 1$.

### 🎓 Exercise 7: Criticality and Entropy

**Task:**

1.  Implement a function `simulate_avalanche()` to simulate a branching process and return the avalanche size.
2.  Run simulations for a range of branching ratios $\sigma$ (e.g., 0.5 to 1.5).
3.  Calculate the entropy of the avalanche sizes for each $\sigma$.
4.  Plot Entropy vs. Branching Ratio. Does it peak at 1?

**Hints:**

- At each step, the number of new active neurons is `np.random.poisson(active_neurons * sigma)`
- Use `np.histogram` to compute the probability distribution of avalanche sizes
- Remember to filter out zero probabilities before computing entropy
- Entropy formula: $H = -\sum P(x) \log_2 P(x)$


In [ ]:
def simulate_avalanche(sigma: float, max_steps: int = 100, max_size: int = 10000) -> int:
    """
    Simulates a branching process and returns the total avalanche size.

    A branching process models how activity spreads through a neural network.
    Each active neuron probabilistically activates other neurons in the next step.

    Args:
        sigma (float): Branching ratio - average number of neurons activated by each neuron.
            - sigma < 1: Subcritical (activity dies out quickly)
            - sigma = 1: Critical (activity persists, power-law distributed sizes)
            - sigma > 1: Supercritical (activity explodes)
        max_steps (int): Maximum simulation steps to prevent infinite loops.
        max_size (int): Maximum avalanche size to prevent memory issues.

    Returns:
        int: Total number of spikes in the avalanche.

    Examples:
        >>> np.random.seed(42)
        >>> simulate_avalanche(0.5)  # Subcritical - small avalanche
        1
        >>> np.random.seed(123)
        >>> simulate_avalanche(0.5)  # Another subcritical example
        2
        >>> np.random.seed(42)
        >>> simulate_avalanche(0.0)  # Zero branching = just initial spike
        1
    """
    ################################
    # YOUR CODE HERE
    # 1. Start with 1 active neuron, total_size = 1
    # 2. Loop until no neurons are active (or max_steps/max_size reached):
    #    - next_active = np.random.poisson(active_neurons * sigma)
    #    - total_size += next_active
    #    - active_neurons = next_active
    # 3. Return total_size
    ################################
    pass


# Test your implementation
doctest.run_docstring_examples(simulate_avalanche, globals())

# Simulation Parameters
sigmas = np.linspace(0.5, 1.5, 11)  # 11 values from subcritical to supercritical
n_trials_avalanche = 500  # Number of avalanche simulations per sigma
entropies = []

################################
# YOUR CODE HERE
# For each sigma in sigmas:
#   1. Run n_trials_avalanche simulations
#   2. Collect avalanche sizes
#   3. Compute probability distribution using np.histogram
#   4. Calculate entropy of the distribution
#   5. Append to entropies list
################################

# Plot (uncomment after implementing)
# fig = px.line(x=sigmas, y=entropies, labels={"x": "Branching Ratio (σ)", "y": "Entropy (bits)"})
# fig.add_vline(x=1.0, line_dash="dash", line_color="red", annotation_text="Critical Point")
# fig.show()

## 11. Conclusion

In this tutorial, we have journeyed from basic spike train statistics to advanced concepts in NeuroAI.

- **Variability**: We learned that the brain is noisy (high CV, Fano Factor), but this noise might be a feature, not a bug.
- **Information**: We quantified how much information spikes carry using Entropy and Mutual Information.
- **Decoding**: We used Machine Learning to "read the mind" of our simulated neurons.
- **Representations**: We explored how biological features (thresholds, latency, population vectors) map to artificial neural network concepts (ReLU, Embeddings).
- **Criticality**: Finally, we saw how operating at the "edge of chaos" maximizes information capacity.

These tools form the foundation for analyzing both biological neural data and the internal representations of artificial agents.


## 12. Further Reading

### Criticality in the Brain

In Section 10.5, we demonstrated that systems at a **critical point** maximize their information capacity (entropy). This is a robust finding in statistical physics and neuroscience.

- **Read more**: [Beggs & Plenz (2003) - Neuronal Avalanches in Neocortical Circuits](https://www.jneurosci.org/content/23/35/11167)
- **Review**: [The Critical Brain (2014)](https://www.frontiersin.org/articles/10.3389/fncir.2014.00054/full)

### Spiking Neural Networks (SNNs)

If you are interested in exploring how population coding is used in Spiking Neural Networks (SNNs) for deep learning, check out this tutorial from the **snntorch** library:

- [**Population Coding in Spiking Neural Networks**](https://snntorch.readthedocs.io/en/latest/tutorials/tutorial_pop.html)

It demonstrates how using a population of neurons to encode input data can improve the performance of SNNs compared to single-neuron encoding.
